In [ ]:
import os
import warnings
import logging

In [ ]:
os.environ.setdefault('TF_CPP_MIN_LOG_LEVEL', '2')  

In [ ]:
warnings.filterwarnings(
    "ignore",
    message=r"Do not pass an `input_shape`/`input_dim` argument to a layer",
    category=UserWarning,
)

In [ ]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, regularizers
from keras.datasets import imdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 1. Load & Preprocess Data 
VOCAB_SIZE = 10000
BATCH_SIZE = 512
EPOCHS = 20
VALIDATION_SPLIT = 0.2

In [ ]:
def load_data(vocab_size):
    (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size)

    def vectorize(sequences):
        result = np.zeros((len(sequences), vocab_size))
        for i, seq in enumerate(sequences):
            result[i, seq] = 1.0
        return result

    x_train = vectorize(train_data)
    x_test = vectorize(test_data)
    y_train = np.asarray(train_labels).astype("float32")
    y_test = np.asarray(test_labels).astype("float32")
    return x_train, y_train, x_test, y_test

In [ ]:
# load and vectorize
x_train, y_train, x_test, y_test = load_data(VOCAB_SIZE)

In [ ]:
# 2. Model Builder
def build_model(units=16, 
                layers_count=2, 
                activation="relu", 
                loss_fn="binary_crossentropy",
                use_dropout=False, 
                dropout_rate=0.5,
                use_l2=False, 
                l2_lambda=0.001):
    model = keras.Sequential()
    for i in range(layers_count):
        if i == 0:
            if use_l2:
                model.add(layers.Dense(units, activation=activation, input_shape=(VOCAB_SIZE,),
                                       kernel_regularizer=regularizers.l2(l2_lambda)))
            else:
                model.add(layers.Dense(units, activation=activation, input_shape=(VOCAB_SIZE,)))
        else:
            if use_l2:
                model.add(layers.Dense(units, activation=activation,
                                       kernel_regularizer=regularizers.l2(l2_lambda)))
            else:
                model.add(layers.Dense(units, activation=activation))
        if use_dropout:
            model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="rmsprop", loss=loss_fn, metrics=["accuracy"])
    return model

In [ ]:
# 3. Experiment Runner
def run_experiment(config_name, units=16, layers_count=2, activation="relu", loss_fn="binary_crossentropy",
                   use_dropout=False, use_l2=False):
    model = build_model(units=units, layers_count=layers_count,
                        activation=activation, loss_fn=loss_fn,
                        use_dropout=use_dropout, use_l2=use_l2)
    history = model.fit(
        x_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=VALIDATION_SPLIT,
        verbose=0
    )
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return {
        "Configuration": config_name,
        "Validation Accuracy": max(history.history.get("val_accuracy", [])),
        "Test Accuracy": test_acc
    }

In [ ]:
# 4. Run Experiments and collect results 
results = []

In [ ]:
# 1 Vary number of hidden layers
results.append(run_experiment("Baseline (2 layers, 16 units, relu, BCE)", units=16, layers_count=2, activation="relu", loss_fn="binary_crossentropy"))
results.append(run_experiment("1 hidden layer, 32 units, relu, BCE", units=32, layers_count=1, activation="relu", loss_fn="binary_crossentropy"))
results.append(run_experiment("3 hidden layers, 16 units, relu, BCE", units=16, layers_count=3, activation="relu", loss_fn="binary_crossentropy"))

In [ ]:
# 2 Vary number of units
results.append(run_experiment("2 layers, 64 units, relu, BCE", units=64, layers_count=2, activation="relu", loss_fn="binary_crossentropy"))
results.append(run_experiment("2 layers, 128 units, relu, BCE", units=128, layers_count=2, activation="relu", loss_fn="binary_crossentropy"))

In [ ]:
# 3. Use MSE loss
results.append(run_experiment("2 layers, 16 units, relu, MSE", units=16, layers_count=2, activation="relu", loss_fn="mse"))

In [ ]:
# 4. Use tanh activation
results.append(run_experiment("2 layers, 16 units, tanh, BCE", units=16, layers_count=2, activation="tanh", loss_fn="binary_crossentropy"))

In [ ]:
# 5. Use Dropout
results.append(run_experiment("2 layers, 16 units, relu, w/ Dropout", units=16, layers_count=2, activation="relu", loss_fn="binary_crossentropy", use_dropout=True))

In [ ]:
# 6. Use Dropout + L2 Regularization
results.append(run_experiment("2 layers, 16 units, relu, Dropout + L2", units=16, layers_count=2, activation="relu", loss_fn="binary_crossentropy", use_dropout=True, use_l2=True))

In [ ]:
df_results = pd.DataFrame(results)
df_sorted = df_results.sort_values(by='Test Accuracy', ascending=True)

In [ ]:
plt.figure(figsize=(10, 6))
plt.barh(df_sorted['Configuration'], df_sorted['Test Accuracy'], color='skyblue')
plt.xlabel("Test Accuracy")
plt.title("IMDB Model Performance Across Configurations")
plt.xlim(0.8, 0.9)
plt.grid(axis='x')
plt.tight_layout()
plt.show()

In [ ]:
print("\nFinal Results Table")
print(df_results.sort_values(by='Test Accuracy', ascending=False).to_string(index=False))